#  Import Statements

In [1]:
import pandas as pd

# Loading Data

In [ ]:
df = pd.read_csv('data/data.csv')
cpi = pd.read_csv('data/cpi.csv')


df.head()
cpi.head()

,"(Quarter, Change from previous quarter (%))",Annual change (%)
Dec-14,0.2,1.7
Mar-15,0.2,1.3
Jun-15,0.7,1.5
Sep-15,0.5,1.5
Dec-15,0.4,1.7


# Preprocessing
## What needs to be done?
- Price adjusted for inflation
- one hot encoding?
- train, val, test split

In [15]:
# Price adjustment for inflation
df['BCreatedDateAEST'] = pd.to_datetime(df['BCreatedDateAEST'])
df['quarter'] = df['BCreatedDateAEST'].dt.to_period('Q')

cpi['Quarter'] = pd.to_datetime(cpi['Quarter'], format='%b-%y')

# Show the result
df.head()

KeyError: 'Quarter'